## xcube Data Store Framework - CCI Open Data Portal

*Please checkout the general introduction to xcube data stores in the Jupyter Notebook [Getting Started](./1_getting_started.ipynb) before jumping into this notebook :)* 

This notebook provides a walk-through demonstrating how to use xcube and the xcube plugin for the [Open Data Portal](https://climate.esa.int/en/odp/#/dashboard) of the [ESA Climate Change Initiative](https://climate.esa.int/en/esa-climate/esa-cci/) (CCI) to read and explore CCI data.


In order to run this notebook you need to install the `xcube_cci` plugin. You may do so by executing the following line in your terminal: 

```
$  conda install -c conda-forge xcube-cci
```

Or you can install `xcube_cci` from sources by following the instructions on https://github.com/dcs4cop/xcube-cci.


In [ ]:
# mandatory imports
from xcube.core.store import find_data_store_extensions
from xcube.core.store import get_data_store_params_schema
from xcube.core.store import new_data_store

# Utilities for notebook visualization
import shapely.geometry
from IPython.display import JSON
import matplotlib.pyplot as plt

Configure matplotlib to display graphs inline directly in the notebook and set a sensible default figure size.

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = 16,8

Check whether the `cciodp` store is among the available stores, if not please follow the installation information from the top of this notebook. 

In [ ]:
JSON({e.name: e.metadata for e in find_data_store_extensions()})

Usually we need more information to get the actual data store object. Which data store parameters are available for `cciodp`? (We'll take a look at the `ccizarr` data store later.)

In [ ]:
get_data_store_params_schema('cciodp')

Provide mandatory parameters to instantiate the store class:

In [ ]:
store = new_data_store('cciodp')
store

Which datasets are provided? (the list may contain both gridded and vector datasets):

We may ask for a specific dataset ...

In [ ]:
store.has_data('esacci.OC.5-days.L3S.CHLOR_A.multi-sensor.multi-platform.MERGED.3-1.geographic')

... but in many cases we want to query for certain criteria. How can we do that?

In [ ]:
store.get_search_params_schema()

Now search, let's search for sea surface temperature data (SST) with a daily frequency: 

In [ ]:
iterator = store.search_data(cci_attrs=dict(ecv='SST', frequency='day'))
JSON([item.to_dict() for item in iterator])

Which parameters must be passsed or are available to open the dataset? 

In [ ]:
store.get_open_data_params_schema('esacci.SST.day.L4.SSTdepth.multi-sensor.multi-platform.OSTIA.1-1.r1')

There are no required parameters, so we can decide what parameters we would like to provide them to open a dataset:

In [ ]:
dataset = store.open_data('esacci.SST.day.L4.SSTdepth.multi-sensor.multi-platform.OSTIA.1-1.r1', 
                          variable_names=['analysed_sst'],
                          time_range=['2008-07-01','2008-07-10'])

dataset

Plot one time stamp of the dataset for a analysed_sst in order to take a brief look at the dataset:  

In [ ]:
dataset.analysed_sst.isel(time=0).plot.imshow(cmap='plasma')

If you take a look at the available data stores above, you may have noticed the data store `ccizarr`. 
This store hosts a selected set of ESA CCI ECVs in the JASMIN Object Storage using the Zarr format.
The datasets in this data store have a much better data access performance than datasets in `cciodp`.
When accessed within the [JASMIN infrastructure](), access speed may increase by another magnitude.

However, the dataset versions in this store may be outdated compared to the data in `cciodp`.
So please make sure, the version of a certain dataset satifies your needs.

The `ccizarr` data store is used in eactly the same way as `cciodp`:

In [ ]:
zarr_store = new_data_store('ccizarr')

In [ ]:
JSON(zarr_store.list_data_ids())

In [ ]:
dataset = zarr_store.open_data("ESACCI-L3C_CLOUD-CLD_PRODUCTS-AVHRR_NOAA-1982-2016-fv3.0.zarr")
dataset

In [ ]:
dataset.cfc.isel(time=0).plot.imshow(cmap='bone')